In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import exists
from os import remove
import json
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import sys
sys.path.insert(0, "../util/")
import util as util

%autosave 5

Autosaving every 5 seconds


# Adding dummies and train-test-split

In [2]:
datingTrainWithDummiesExists = exists('../data/datingTrainWithDummies.csv')
datingTestWithDummiesExists = exists('../data/datingTestWithDummies.csv')
datingTrainWithoutDummiesExists = exists('../data/datingTrainWithoutDummies.csv')
datingTestWithoutDummiesExists = exists('../data/datingTestWithoutDummies.csv')
datingFullWithDummiesExists = exists('../data/datingFullWithDummies.csv')
datingFullWithoutDummiesExists = exists('../data/datingFullWithoutDummies.csv')

relatedDummiesDictionaryExists = exists('../data/relatedDummiesDictionary.json')

In [3]:
with open('../data/columnDataDictionary.json') as d:
    columnDataDictionary = json.load(d)
columnList = columnDataDictionary['columnList']
nonBinaryCategoricalList = columnDataDictionary['nonBinaryCategoricalList']
stringToFloatList = columnDataDictionary['stringToFloatList']
pointDistributionList = columnDataDictionary['pointDistributionList']
partnerList = columnDataDictionary['partnerList']

In [4]:
if (datingTrainWithDummiesExists and datingTestWithDummiesExists and datingFullWithDummiesExists and
    datingTrainWithoutDummiesExists and datingTestWithoutDummiesExists and datingFullWithoutDummiesExists and
    relatedDummiesDictionaryExists):
    datingTrainWithDummies = pd.read_csv('../data/datingTrainWithDummies.csv')
    datingTestWithDummies = pd.read_csv('../data/datingTestWithDummies.csv')
    datingTrainWithoutDummies = pd.read_csv('../data/datingTrainWithoutDummies.csv')
    datingTestWithoutDummies = pd.read_csv('../data/datingTestWithoutDummies.csv')
    datingFullWithDummies = pd.read_csv('../data/datingFullWithDummies.csv')
    datingFullWithoutDummies = pd.read_csv('../data/datingFullWithoutDummies.csv')
    
    with open('../data/relatedDummiesDictionary.json') as d:
        relatedDummiesDictionary = json.load(d)
    for df in [datingTrainWithDummies,datingTestWithDummies,datingFullWithDummies,datingTrainWithoutDummies,datingTestWithoutDummies,datingFullWithoutDummies]:
        df['zipcode'] = df['zipcode'].apply(str)
        if 'zipcode_o' in list(df.columns):
            df['zipcode_o'] = df['zipcode_o'].apply(str)
        for col in nonBinaryCategoricalList:
            if col in list(df.columns):
                df[col] = df[col].apply(str)
        
else:
    if (datingTrainWithDummiesExists):
        remove('../data/datingTrainWithDummies.csv')
    if (datingTestWithDummiesExists):
        remove('../data/datingTestWithDummies.csv')
    if (datingTrainWithoutDummiesExists):
        remove('../data/datingTrainWithoutDummies.csv')
    if (datingTestWithoutDummiesExists):
        remove('../data/datingTestWithoutDummies.csv')
    if (datingFullWithDummiesExists):
        remove('../data/datingFullWithDummies.csv')
    if (datingFullWithoutDummiesExists):
        remove('../data/datingFullWithoutDummies.csv')
    if (relatedDummiesDictionaryExists):
        remove('../data/relatedDummiesDictionary.json')
    
    !jupyter nbconvert --execute eda.ipynb
    
    datingData = pd.read_csv('../data/encoded-SpeedDatingData.csv')
    blindDateData = datingData[columnList]
    
    for col in stringToFloatList:
        blindDateData[col] = blindDateData[col].str.replace(',', '').astype(float)
    blindDateData['zipcode'] = blindDateData['zipcode'].str.replace(',', '')
    for col in nonBinaryCategoricalList:
        blindDateData[col] = blindDateData[col].apply(str)
    
    blindDateCategoricalData = blindDateData.select_dtypes(include=['O'])
    for col in blindDateCategoricalData.columns:
        blindDateData[col]=blindDateData[col].fillna('nan')
    relatedDummiesDictionary = {}
    for col in blindDateCategoricalData.columns:
        dummyData = pd.get_dummies(blindDateData[col],prefix=col,drop_first=True)
        for dummyCol in dummyData.columns:
            relatedDummiesDictionary[str(dummyCol)] = list(dummyData.columns)
            if col in partnerList:
                partnerList.append(str(dummyCol))
        blindDateData = pd.concat([blindDateData,dummyData],axis=1)
    with open('../data/relatedDummiesDictionary.json', 'w') as fp:
        json.dump(relatedDummiesDictionary, fp)
        
    partnerList = list(set(partnerList))
    columnDataDictionary = {"columnList": columnList,
                        "nonBinaryCategoricalList": nonBinaryCategoricalList,
                        "stringToFloatList": stringToFloatList,
                        "pointDistributionList": pointDistributionList,
                        "partnerList": partnerList}

    with open('../data/columnDataDictionary.json', 'w') as fp:
            json.dump(columnDataDictionary, fp)
    
    datingFullWithDummies = blindDateData.copy()
    match = datingFullWithDummies['match']
    X = datingFullWithDummies.drop(['match'], axis=1)
    
    datingTrainWithDummies, datingTestWithDummies, matchTrain, matchTest = train_test_split(X, match, test_size=0.2)
    
    datingTrainWithDummies['match'] = matchTrain
    datingTestWithDummies['match'] = matchTest
    
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTestWithDummies.to_csv('../data/datingTestWithDummies.csv',index=False)
    datingFullWithDummies.to_csv('../data/datingFullWithDummies.csv',index=False)
    
    dummyColumns = list(relatedDummiesDictionary.keys())
    datingTrainWithoutDummies = datingTrainWithDummies.drop(dummyColumns, axis=1)
    datingTestWithoutDummies = datingTestWithDummies.drop(dummyColumns, axis=1)
    datingFullWithoutDummies = datingFullWithDummies.drop(dummyColumns, axis=1)
    
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)
    datingTestWithoutDummies.to_csv('../data/datingTestWithoutDummies.csv',index=False)
    datingFullWithoutDummies.to_csv('../data/datingFullWithoutDummies.csv',index=False)

/Users/garysimmons/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Data Cleaning

### Join partner

In [5]:
for partnerCol in partnerList:
    if (("_o" not in str(partnerCol)) and (partnerCol+"_o" not in datingTrainWithoutDummies.columns) and (partnerCol+"_o" not in datingTrainWithDummies.columns)):
        partnerWithDummies = datingFullWithDummies.copy()
        partnerWithoutDummies = datingFullWithoutDummies.copy()
        datingTrainWithDummies = util.joinToPartner(datingTrainWithDummies,partnerWithDummies)
        datingTrainWithoutDummies = util.joinToPartner(datingTrainWithoutDummies,partnerWithoutDummies)
        datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
        datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)
        break

### Get distance

In [6]:
if (("partnerDistance" not in datingTrainWithoutDummies.columns) or 
    ("partnerDistance" not in datingTrainWithDummies.columns)):
    datingTrainWithoutDummies = util.returnDFWithpartnerDistance(datingTrainWithoutDummies,"train",True)
    datingTrainWithDummies["partnerDistance"] = datingTrainWithoutDummies["partnerDistance"]
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)

### Fix ambiguous scores

In [9]:
halfwayChangeColumnsFromWithDummies = [str(col) for col in datingTrainWithDummies.columns if (("1_s" in str(col)) | ("3_s" in str(col)))]
halfwayChangeColumnsFromWithoutDummies = [str(col) for col in datingTrainWithoutDummies.columns if (("1_s" in str(col)) | ("3_s" in str(col)))]

if(len(halfwayChangeColumnsFromWithDummies) + len(halfwayChangeColumnsFromWithoutDummies) > 0):
    for question in pointDistributionList:
        questionCols = [str(col) for col in datingTrainWithDummies if question in str(col)]
        datingTrainWithDummies[f'{question}_sum'] = datingTrainWithDummies[questionCols].sum(axis = 1)
        datingTrainWithoutDummies[f'{question}_sum'] = datingTrainWithoutDummies[questionCols].sum(axis = 1)
        for questionCol in questionCols:
            datingTrainWithDummies[str(questionCol)] = 0 if datingTrainWithDummies[f'{question}_sum'] == 0 else datingTrainWithDummies[str(questionCol)] * 100 / datingTrainWithDummies[f'{question}_sum']
            datingTrainWithoutDummies[str(questionCol)] = 0 if datingTrainWithoutDummies[f'{question}_sum'] == 0 else datingTrainWithoutDummies[str(questionCol)] * 100 / datingTrainWithoutDummies[f'{question}_sum']
    for halfwayQuestion in halfwayChangeColumnsFromWithDummies:
        targetQuestion = ""
        if ("1_s" in halfwayQuestion):
            if halfwayQuestion == "attr1_s":
                targetQuestion = "pf_o_att"
            elif halfwayQuestion == "sinc1_s":
                targetQuestion = "pf_o_sin"
            elif halfwayQuestion == "intel1_s":
                targetQuestion = "pf_o_int"
            elif halfwayQuestion == "fun1_s":
                targetQuestion = "pf_o_fun"
            elif halfwayQuestion == "amb1_s":
                targetQuestion = "pf_o_amb"
            elif halfwayQuestion == "shar1_s":
                targetQuestion = "pf_o_sha"
            else:
                targetQuestion = halfwayQuestion.replace("1_s","1_1")
        else:
            targetQuestion = halfwayQuestion.replace("3_s","3_1")
        currentMindsetAnswers = []
        for rowindex in range(datingTrainWithDummies):
            row = datingTrainWithDummies.iloc[rowindex]
            if (util.isNan(row[halfwayQuestion]) | row["order"] <= int(row["round"])):
                currentMindsetAnswers.append(row[targetQuestion])
            else:
                currentMindsetAnswers.append(row[halfwayQuestion])
        datingTrainWithDummies[targetQuestion] = pd.Series(currentMindsetAnswers)
        datingTrainWithoutDummies[targetQuestion] = pd.Series(currentMindsetAnswers)
        datingTrainWithDummies.drop(halfwayQuestion,axis = 1)
        datingTrainWithoutDummies.drop(halfwayQuestion,axis = 1)
    datingTrainWithDummies.to_csv('../data/datingTrainWithDummies.csv',index=False)
    datingTrainWithoutDummies.to_csv('../data/datingTrainWithoutDummies.csv',index=False)

KeyError: "['from_Seattle_o', 'from_Seattle', 'undergra_marymount manhattan college_o', 'undergra_marymount manhattan college', 'from_Manhattan_o', 'from_Manhattan'] not in index"

### Replace Nans

# Training

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Individual Testing

## Logistic Regression

## KNN

## Random Forest Classifier

## Boosting

# Ensemble Testing